Topic: Project 5  
Subject: Getting Track Lyrics from Lyrics Wikia   
Date: 12/12/2017  
Name: Zach Heick  

**Summary**: Uses PyLyrics library to scrape lyrics and store in PostgreSQL database.

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, select
import os
import boto3
from PyLyrics import *
import pickle
import time

In [2]:
engine_name = 'postgresql://' + str(os.environ['POSTGRESQL_U']) + ':' + str(os.environ['POSTGRESQL_P']) + '@13.59.54.149:5432/project_kojak'
engine = create_engine(engine_name)
df = pd.read_sql_query('''SELECT * FROM tracks JOIN artists ON tracks.artist_id = artists.artist_id
                          WHERE tracks.energy IS NOT NULL;''', engine)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141311 entries, 0 to 141310
Data columns (total 15 columns):
album_art            141311 non-null object
album_id             141311 non-null object
album_name           141311 non-null object
artist_id            141311 non-null object
duration_ms          141311 non-null int64
energy               141311 non-null float64
liveness             141311 non-null float64
loudness             141311 non-null float64
speechiness          141311 non-null float64
tempo                141311 non-null float64
track_id             141311 non-null object
track_name           141311 non-null object
artist_name          141311 non-null object
artist_id            141311 non-null object
artist_popularity    141311 non-null int64
dtypes: float64(5), int64(2), object(8)
memory usage: 16.2+ MB


In [4]:
df.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98815 entries, 0 to 141310
Data columns (total 15 columns):
album_art            98815 non-null object
album_id             98815 non-null object
album_name           98815 non-null object
artist_id            98815 non-null object
duration_ms          98815 non-null int64
energy               98815 non-null float64
liveness             98815 non-null float64
loudness             98815 non-null float64
speechiness          98815 non-null float64
tempo                98815 non-null float64
track_id             98815 non-null object
track_name           98815 non-null object
artist_name          98815 non-null object
artist_id            98815 non-null object
artist_popularity    98815 non-null int64
dtypes: float64(5), int64(2), object(8)
memory usage: 12.1+ MB


In [6]:
s3 = boto3.client('s3', 
                  aws_access_key_id=os.environ['AWS_ACCESS_KEY'], 
                  aws_secret_access_key=os.environ['AWS_SECRET_KEY'])

In [ ]:
lyrics = []
for i, row in enumerate(df[['artist_name', 'track_id', 'track_name']].itertuples()):
    if i % 100 == 0 and i != 0:
        print(i, end=' ')
        
    track_data = {}
    track_data['artist_name'] = row[1]
    track_data['track_id'] = row[2]
    track_data['track_name'] = row[3]
    
    # Get lyrics from Lyrics Wiki, otherwise None
    try:
        track_data['lyrics'] = PyLyrics.getLyrics(row[1], row[3])
        lyrics.append(track_data)
        time.sleep(0.4)
    except ValueError:
        track_data['lyrics'] = None
        lyrics.append(track_data)
  
# Store Data
with open('lyrics.pickle', 'wb') as f:
    pickle.dump(lyrics, f)

s3.upload_file('lyrics.pickle', 
               'metis-project-kojak-bucket', 
               'lyrics_data/lyrics.pickle')
m = MetaData()
m.reflect(engine)
conn = engine.connect()
conn.execute(m.tables['lyrics'].insert(), lyrics)